# Enterprise Data Science_COVID-19

### Author: Akanksha Parashar
### Matrikelnummer: 423022

## Notebook Description
- In this notebook, we are trying to use 'Groupby' command to filter out the data.
- Befor applying groupby to the entire dataset, the groupby functionality should be tested on a small test dataset.
- The dataset of the COVID infected population is taken from John Hopkins COVID-19 GitHub page.
- The datapath given in this notebook is of user's computer folder where the required csv files are saved.
- The data source is given in the Readme file.

In [34]:
# importing the libraries
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Groupby apply on large (relational) data set
# Attentions all writen functions assume a data frame where the date is sorted!!

In [35]:
pd_JH_data=pd.read_csv('C:/Users/hp/Documents/ads_covid_19/data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,Kosovo,0
2,2020-01-22,no,Kuwait,0
3,2020-01-22,no,Kyrgyzstan,0
4,2020-01-22,no,Laos,0


# Test data

In [36]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [37]:
test_data.head()

,date,state,country,confirmed
16854,2020-03-21,no,Germany,16662
16904,2020-03-21,no,US,26025
17139,2020-03-22,no,Germany,18610
17203,2020-03-22,no,US,34944
17424,2020-03-23,no,Germany,22672


In [38]:
test_data.groupby(['country']).agg(np.max)
# np.max show the max. confirmed cases in each country

,date,state,confirmed
country,,,
Germany,2022-06-16,no,27124689
US,2022-06-16,no,86057735


In [39]:
test_data.groupby(['state','country']).agg(np.max)
# state and country both are index here.

date  confirmed
state country                      
no    Germany 2022-06-16   27124689
      US      2022-06-16   86057735

In [40]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    #assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [41]:
test_data.groupby(['state','country']).agg(np.max)
# state and country both are index here.

date  confirmed
state country                      
no    Germany 2022-06-16   27124689
      US      2022-06-16   86057735

In [42]:
# groupby doesn't work on numpy array that is why we have removed the if part from the build_feature file...
#...we have to apply groupby on pandas dataframe

In [43]:
#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [44]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [45]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [46]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Germany  16854             NaN
                17139             NaN
                17424        6.427510
                17709        5.190573
                17994        6.129400
                             ...     
       US       248609    4004.504719
                248908    1067.778780
                249186     704.961475
                249464     624.258667
                249761     574.281917
Name: confirmed, Length: 1636, dtype: float64

In [47]:
# the column after country is an important column (index column of the test_data) to bring back the.... 
#...information in the last column to our larger dataset

In [48]:
test_data

,date,state,country,confirmed
16854,2020-03-21,no,Germany,16662
16904,2020-03-21,no,US,26025
17139,2020-03-22,no,Germany,18610
17203,2020-03-22,no,US,34944
17424,2020-03-23,no,Germany,22672
...,...,...,...,...
249186,2022-06-14,no,US,85758512
249415,2022-06-15,no,Germany,27096571
249464,2022-06-15,no,US,85941290
249699,2022-06-16,no,Germany,27124689


In [49]:
# now we have to merge both the test_data and regression data on the basis of index value...now we will do it directly on large...
#...dataset

In [50]:
# pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

In [51]:
pd_DR_result = pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [52]:
pd_DR_result.head()

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [53]:
pd_DR_result = pd_DR_result.rename(columns = {'confirmed' : 'doubling_rate', 'level_2' : 'index'})
pd_DR_result.head()

,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [59]:
pd_DR_result.columns

Index(['state', 'country', 'index', 'doubling_rate'], dtype='object')

In [54]:
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,Kosovo,0
2,2020-01-22,no,Kuwait,0
3,2020-01-22,no,Kyrgyzstan,0
4,2020-01-22,no,Laos,0


In [60]:
pd_JH_data['index'] = pd_JH_data.index
pd_JH_data.head()

,date,state,country,confirmed,index
0,2020-01-22,Alberta,Canada,0,0
1,2020-01-22,no,Kosovo,0,1
2,2020-01-22,no,Kuwait,0,2
3,2020-01-22,no,Kyrgyzstan,0,3
4,2020-01-22,no,Laos,0,4


In [61]:
pd_JH_data.columns

Index(['date', 'state', 'country', 'confirmed', 'index'], dtype='object')

In [55]:
#pd_JH_data.reset_index().head()
# as john hopkins dataset doesn't have index so we added index

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,Kosovo,0
2,2,2020-01-22,no,Kuwait,0
3,3,2020-01-22,no,Kyrgyzstan,0
4,4,2020-01-22,no,Laos,0


In [58]:
pd_JH_data.columns

Index(['date', 'state', 'country', 'confirmed'], dtype='object')

In [56]:
# now we will merge the doubling rate and john hopkins dataset on the basis of index column

In [62]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result,on=['index'],how='left')
pd_result_larg.head()

,date,state_x,country_x,confirmed,index,state_y,country_y,doubling_rate
0,2020-01-22,Alberta,Canada,0,0,Alberta,Canada,NaN
1,2020-01-22,no,Kosovo,0,1,no,Kosovo,NaN
2,2020-01-22,no,Kuwait,0,2,no,Kuwait,NaN
3,2020-01-22,no,Kyrgyzstan,0,3,no,Kyrgyzstan,NaN
4,2020-01-22,no,Laos,0,4,no,Laos,NaN


In [63]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','doubling_rate']],on=['index'],how='left')
pd_result_larg.head()

,date,state,country,confirmed,index,doubling_rate
0,2020-01-22,Alberta,Canada,0,0,NaN
1,2020-01-22,no,Kosovo,0,1,NaN
2,2020-01-22,no,Kuwait,0,2,NaN
3,2020-01-22,no,Kyrgyzstan,0,3,NaN
4,2020-01-22,no,Laos,0,4,NaN


In [64]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply

In [65]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [66]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [67]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,date,state,country,confirmed,index,doubling_rate,confirmed_filtered
0,2020-01-22,Alberta,Canada,0,0,NaN,0.0
1,2020-01-22,no,Kosovo,0,1,NaN,0.0
2,2020-01-22,no,Kuwait,0,2,NaN,0.0
3,2020-01-22,no,Kyrgyzstan,0,3,NaN,0.0
4,2020-01-22,no,Laos,0,4,NaN,0.0


# Filtered doubling rate

In [68]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
249940,no,Zimbabwe,248661,3377.909574
249941,no,Zimbabwe,248874,3471.211293
249942,no,Zimbabwe,249181,3073.432487
249943,no,Zimbabwe,249423,2565.718130
249944,no,Zimbabwe,249944,2445.801923


In [69]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,date,state,country,confirmed,index,doubling_rate,confirmed_filtered,confirmed_filtered_DR
249940,2022-06-16,no,Andorra,43449,249940,3.855467e+02,43449.0,642.688889
249941,2022-06-16,no,Algeria,265964,249941,2.532886e+04,265962.4,29882.415731
249942,2022-06-16,no,Albania,276821,249942,inf,276839.0,10252.296296
249943,2022-06-16,no,Argentina,9313453,249943,inf,9320820.0,842.474956
249944,2022-06-16,no,Zimbabwe,254502,249944,1.465983e+03,254467.4,2445.801923


In [70]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
# mask is a true or false vector

In [71]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,date,state,country,confirmed,index,doubling_rate,confirmed_filtered,confirmed_filtered_DR
248559,2022-06-12,no,Germany,26809245,248559,9968.635056,26867898.6,512.391430
248844,2022-06-13,no,Germany,26915085,248844,482.704820,26926439.4,478.442701
249130,2022-06-14,no,Germany,27007429,249130,271.571735,26990603.8,438.910722
249415,2022-06-15,no,Germany,27096571,249415,297.613719,27071841.2,371.333708
249699,2022-06-16,no,Germany,27124689,249699,461.815277,27153078.6,333.243570


In [72]:
pd_result_larg.to_csv('C:/Users/hp/Documents/ads_covid_19/data/processed/COVID_final_set.csv',sep=';',index=False)